# Simple Multi-Layer Perceptron

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

datapath = 'data'

# load dataset
data_train = MNIST(
    root = datapath,
    train = True,                         
    transform = ToTensor(), 
)
data_test = MNIST(
    root = datapath, 
    train = False, 
    transform = ToTensor(),
)

### Important hyperparameters

- *batch size*: Determines on how many samples the gradient is computed at each iteration. A large batch typically implies a more stable training but may also lead the model to a bad local minimum.

- *learning rate*: A scalar value that is multiplied by the gradient. If the learning rate is too large, the model may not converge. Conversely, a small learning rate may cause a very slow training and sometimes the model may get "stuck" on local minima.

- *number of epochs*: Determines how many time the model is trained on the whole dataset. Training for too many epochs can lead to overfitting but there's an easy fix for that (see `02c_MLP_GPU.ipynb`).

In general, you wouldn't like your architecture to be too sensitive to these parameters.
For example, if you change the learning rate from $10^{-3}$ to $1$, indeed, you should expect a major change.
However, if you change the learning rate from $0.001$ to $0.0011$ and your model is not learning anymore, probably you should change the architecture.

In [ ]:
# hyperparameters
batch_size = 16
lr = 1e-3
n_epochs = 10

In [ ]:
print("Size of training dataset:", len(data_train))
print("Size of test dataset:", len(data_test))

In [ ]:
x_sample, y_sample = data_train[0]

print("Shape of x_sample:", x_sample.shape)
print("y_sample is an integer:", type(y_sample))

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(x_sample[0], cmap='binary')
plt.title(f'Label: {y_sample: d}')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
import torch.nn as nn

class SimpleMLP(nn.Module):

    """
    - input_shape: shape of a single input data point
    """
    def __init__(self, input_shape, n_classes):
        super(SimpleMLP, self).__init__()
        self.input_shape = np.asarray(input_shape)
        self.n_classes = n_classes
        self.seq_model = nn.Sequential(
            nn.Linear(self.input_shape.prod(), 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, self.n_classes)
        )

    def forward(self, x):
        x = x.view(-1, self.input_shape.prod())  # make the input of shape (batch_size, height*weight)
        logits = self.seq_model(x)
        return logits
    
    # def __repr__(self):
    #     return "Overwritten print"


In [ ]:
model = SimpleMLP(input_shape=(1, 28, 28), n_classes=10)
print(model)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data_test, batch_size=32, shuffle=False)

In [ ]:
x_batch, y_batch = next(iter(train_loader))
print("x_batch shape:", x_batch.shape)
print("y_batch shape:", y_batch.shape)

In [ ]:
pred_batch = model(x_batch)
print("Example of model's logits shape:", pred_batch.shape)

In [ ]:
def model_accuracy(data_loader):
    n_total = 0
    n_correct = 0

    for x_batch, y_batch in data_loader:
        logits_batch = model(x_batch)  # model's output scores
        n_total += len(y_batch)
        n_correct += sum(logits_batch.argmax(axis=-1) == y_batch)
    return (n_correct / n_total).item()

print(f"Train accuracy before training: {model_accuracy(train_loader):.4f}")
print(f"Test accuracy before training: {model_accuracy(test_loader):.4f}")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from tqdm import tqdm

for _ in tqdm(range(n_epochs)):
    for x_batch, y_batch in train_loader:

        optimizer.zero_grad()
        logits_batch = model(x_batch)
        loss_batch = loss_fn(logits_batch, y_batch)
        loss_batch.backward()
        optimizer.step()

In [ ]:
print(f"Train accuracy before training: {model_accuracy(train_loader):.4f}")
print(f"Test accuracy before training: {model_accuracy(test_loader):.4f}")

### Saving and loading models

There are many ways to save and load models in Pytorch, but the best practice is to use state dictionaries.

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
torch.save(model.state_dict(), "saved_models/MLP.pt")

IMP: If you resume training after saving and loading the model's state dictionary, this is not equal to uninterrupted training.

This is because the state of the optimizer will be reset.
If you want to resume your training, you should also save the state dictionary of your optimizer.